In [1]:
# ZILLOW DATA EXTRACTION WRITTEN BY SONIA YANG

# Dependencies
import requests
import urllib
import random
import math
import pandas as pd
import xml.etree.ElementTree as ET
import time
from config import zws_id, gkey # please use your own Zillow & Google API keys!
from urllib.request import urlopen

In [2]:
# FUNCTION to grab the exact address based on longitude and latitude
# modified from here https://gist.github.com/bradmontgomery/5397472
# their example didn't include an API key, but I added it otherwise you'd hit the rate limit easily

def reverse_geocode(latitude, longitude):
    # Did the geocoding request comes from a device with a
    # location sensor? Must be either true or false
    sensor = 'true'

    # Hit Google's reverse geocoder directly
    # NOTE: I *think* their terms state that you're supposed to
    # use google maps if you use their api for anything.
    base = "https://maps.googleapis.com/maps/api/geocode/json?"
    params = "latlng={lat},{lon}&sensor={sen}&key={key}".format(
        lat=latitude,
        lon=longitude,
        sen=sensor,
        key=gkey
    )
    url = "{base}{params}".format(base=base, params=params)
    #print(url)
    response = requests.get(url).json()
    address = response['results'][0]['formatted_address']
    return address


In [3]:
# FUNCTION to generate random lat & lng within a certain radius 
# modified from here: http://hadoopguru.blogspot.com/2014/12/python-generate-random-latitude-and.html
# changed to take in an empty initial dataframe and load in the data + return it
# this calls the reverse geocode function to grab the addresses of each randomly generated lat & lng

def generate_addresses(latitude, longitude, df):
    
    radius = 10000                         #Choose your own radius
    radiusInDegrees=radius/111300            
    r = radiusInDegrees

    counter = 0
    
    for i in range(1,200):                 #Choose number of Lat Long to be generated

        u = float(random.uniform(0.0,1.0))
        v = float(random.uniform(0.0,1.0))

        w = r * math.sqrt(u)
        t = 2 * math.pi * v
        x = w * math.cos(t) 
        y = w * math.sin(t)

        xLat  = x + latitude
        yLng = y + longitude

        df.set_value(counter, "latitude", xLat)
        df.set_value(counter, "longitude", yLng)
        
        #print(format(counter) + ": " + format(xLat) + ", " + format(yLng))
        address = reverse_geocode(xLat, yLng).split(',')
        citystatezip = address[1] + address[2]
        
        df.set_value(counter, "address", address[0])
        df.set_value(counter, "city_state_zip", citystatezip)
        
        # Add to counter
        counter = counter + 1
    
    return df

In [4]:
# FUNCTION to call Zillow API's GetSearchResults and will check to see if a house exists at that address
# message code will be written to dataframe
# zillow url format
# http://www.zillow.com/webservice/GetSearchResults.htm?zws-id=<ZWSID>&address=2114+Bigelow+Ave&citystatezip=Seattle%2C+WA
        
def get_message_codes(df):

    for index, row in df.iterrows():

        try:
            url = 'https://www.zillow.com/webservice/GetSearchResults.htm?zws-id='
            address = row['address']
            citystatezip =row['city_state_zip']


            query_url = url + zws_id + '&address=' + urllib.parse.quote(address) + '&citystatezip=' + urllib.parse.quote(citystatezip) 
            #print(query_url)

            root = ET.parse(urlopen(query_url)).getroot()

            for message in root.iter('message'):
                message_code = message[1].text

            print(format(index) + ": " + message_code)

            df.set_value(index, 'message_code', message_code)

            time.sleep(0.5) #necessary bc bombarding Zillow with API calls doesn't allow enough time to respond to each

        except:
            break
    

In [5]:
# FUNCTION to call Zillow's GetDeepSearchResults and look up Zestimate, bed, and bath
# http://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=<ZWSID>&address=2114+Bigelow+Ave&citystatezip=Seattle%2C+WA
# there are some limitations such as multiple zestimates depending on when the house was sold/if it was sold multiple times
# the code to handle that would get too convoluted so I am just writing in the most recent (according to the API) values
# probably not what we would do in real life
# but a decision we made re: the scope of a classroom project on a short time constraint

def search_zillow(df):
    
    for index, row in df.iterrows():
        try:
            url = 'https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id='
            address = df['address'][index]
            citystatezip = df['city_state_zip'][index]


            query_url = url + zws_id + '&address=' + urllib.parse.quote(address) + '&citystatezip=' + urllib.parse.quote(citystatezip) 


            root = ET.parse(urlopen(query_url)).getroot()

            print("row " + format(index) + ": " + address + citystatezip)
            print(query_url)

            for zestimate in root.iter('zestimate'):
                zestimate_value = zestimate[0].text

                if zestimate_value is None:
                    print('not for sale')
                else:
                    print ('zestimate (value): ' + format(zestimate[0].text)) 
                    df.set_value(index, 'zestimate', zestimate_value)

            for bedroom in root.iter('bedrooms'):
                bedrooms = bedroom.text
                print("bedrooms: " + bedrooms)
                df.set_value(index, 'zillow_bedrooms', bedrooms)

            for bathroom in root.iter('bathrooms'):
                bathrooms = bathroom.text
                print("bathrooms: " + bathrooms + "\n")
                df.set_value(index, 'zillow_bathrooms', bathrooms)           
            
            print('\n')

            time.sleep(0.5) 


        except:
            break


<h2>HOW TO RUN THIS CODE</h2>
<ol>
<li>Initialize an empty dataframe with the fields as marked below</li>
<li>Call the <strong>generate_addresses</strong> function passing in your empty dataframe</li>
<li>Call the <strong>get_message_codes</strong> to update your dataframe with message codes indicating whether or not a valid property exists at each address. <strong>IMPORTANT:</strong> please register your own Zillow account/get your own key for this!! If we all keep using the same one we'll easily hit the rate limit </li>
<li>Drop the rows in the dataframe for which a property does not exist at that address</li>
<li>Call the <strong>search_zillow</strong> function to get the zestimate (aka price of the property), # of bedrooms, and # of bathrooms</li>
<li>I did not include it in my code, but once you get a sample size of data that you are satisfied with for the city, maybe write it out to a CSV so you don't have to keep running this code/can use it later</li>
</ol>

feel free to comment out my print statements while the functions are running if you find them distracting

In [6]:
# HOW TO RUN ALL THE FUNCTIONS, USING LOS ANGELES AS AN EXAMPLE

# coordinates taken from the CitiesGeo_Output.csv
# we should manually run the following code on each individual city instead of nesting it in another loop
# while this may be hardcoded, it's better than waiting on one gigantic loop that takes forever

# STEP 1: INITALIZE THE DATAFRAME
# if we need any more fields, let me know
la_df = pd.DataFrame({"latitude":'',
                      "longitude":'',
                      "address":'',
                      "city_state_zip":'',
                      "message_code":'',
                      "zestimate":'',
                      "zillow_bedrooms":'',
                      "zillow_bathrooms":''}, index=[0])

# STEP 2: GENERATE RANDOM ADDRESSES IN THE DESIGNATED AREA
# pass in the coordinates for Los Angeles plus the empty dataframe
generate_addresses(34.0522342,-118.2436849, la_df) 

,address,city_state_zip,latitude,longitude,message_code,zestimate,zillow_bathrooms,zillow_bedrooms
0,3801 Sutro Ave,Los Angeles CA 90008,34.0177,-118.324,,,,
1,739 Fremont Villas,Los Angeles CA 90042,34.0965,-118.183,NaN,NaN,NaN,NaN
2,1907 Redcliff St,Los Angeles CA 90039,34.0943,-118.27,NaN,NaN,NaN,NaN
3,1206 W 36th St,Los Angeles CA 90007,34.0227,-118.295,NaN,NaN,NaN,NaN
4,2911-2919 Paola Ave,Los Angeles CA 90032,34.0806,-118.181,NaN,NaN,NaN,NaN
5,235 S Rampart Blvd,Los Angeles CA 90057,34.0679,-118.278,NaN,NaN,NaN,NaN
6,1900 N San Fernando Rd,Los Angeles CA 90065,34.099,-118.237,NaN,NaN,NaN,NaN
7,443 46th St,Los Angeles CA 90011,34.0018,-118.268,NaN,NaN,NaN,NaN
8,554 N Hoover St,Los Angeles CA 90004,34.0808,-118.284,NaN,NaN,NaN,NaN
9,715 E 78th St,Los Angeles CA 90001,33.9686,-118.262,NaN,NaN,NaN,NaN


In [7]:
# STEP 3: CALL THE ZILLOW API TO GET MESSAGE CODES
# 0 means there is a valid property at that address
# 508 and anything else means there isn't
# if you get nothing but invalid message codes, re-run STEP 2
# you might have to sign up for a new Zillow account if you keep getting invalid results here
# there is a possibility you hit the rate limit

get_message_codes(la_df)

0: 0
1: 0
2: 0
3: 0
4: 508
5: 508
6: 508
7: 0
8: 508
9: 0
10: 508
11: 508
12: 508
13: 508
14: 508
15: 0
16: 0
17: 0
18: 0
19: 508
20: 508
21: 508
22: 508
23: 508
24: 0
25: 508
26: 508
27: 508
28: 508
29: 0
30: 508
31: 0
32: 508
33: 0
34: 0
35: 508
36: 508
37: 0
38: 508
39: 508
40: 508
41: 0
42: 508
43: 508
44: 0
45: 0
46: 508
47: 0
48: 508
49: 0
50: 508
51: 508
52: 508
53: 0
54: 508
55: 508
56: 508
57: 508
58: 0
59: 0
60: 0
61: 508
62: 0
63: 0
64: 508
65: 508
66: 508
67: 0
68: 508
69: 0
70: 508
71: 508
72: 0
73: 508
74: 0
75: 0
76: 0
77: 0
78: 508
79: 0
80: 0
81: 0
82: 0
83: 508
84: 0
85: 508
86: 508
87: 0
88: 508
89: 508
90: 508
91: 0
92: 508
93: 508
94: 508
95: 0
96: 508
97: 508
98: 508
99: 0
100: 508
101: 508
102: 508
103: 0
104: 0
105: 508
106: 508
107: 508
108: 0
109: 508
110: 0
111: 508
112: 508
113: 0
114: 0
115: 508
116: 508
117: 0
118: 508
119: 508
120: 0
121: 0
122: 0
123: 508
124: 508
125: 508
126: 508
127: 0
128: 508
129: 0
130: 0
131: 508
132: 508
133: 0
134: 508
135: 0
13

In [8]:
# STEP 4: DROP INVALID ENTRIES FROM DATAFRAME 
# cull all the rows where houses do not exist at the address
# take what is valid (message code of '0')
# the code sometimes might break/not get a response from the server so it's better to take what IS valid

la_df = la_df[la_df.message_code == '0']
la_df

,address,city_state_zip,latitude,longitude,message_code,zestimate,zillow_bathrooms,zillow_bedrooms
0,3801 Sutro Ave,Los Angeles CA 90008,34.0177,-118.324,0,,,
1,739 Fremont Villas,Los Angeles CA 90042,34.0965,-118.183,0,NaN,NaN,NaN
2,1907 Redcliff St,Los Angeles CA 90039,34.0943,-118.27,0,NaN,NaN,NaN
3,1206 W 36th St,Los Angeles CA 90007,34.0227,-118.295,0,NaN,NaN,NaN
7,443 46th St,Los Angeles CA 90011,34.0018,-118.268,0,NaN,NaN,NaN
9,715 E 78th St,Los Angeles CA 90001,33.9686,-118.262,0,NaN,NaN,NaN
15,3027 W 12th Pl,Los Angeles CA 90006,34.0486,-118.307,0,NaN,NaN,NaN
16,3525 E 53rd St,Maywood CA 90270,33.9948,-118.202,0,NaN,NaN,NaN
17,2312 Norwalk Ave,Los Angeles CA 90041,34.1328,-118.219,0,NaN,NaN,NaN
18,4620 Eugene St,East Los Angeles CA 90022,34.0375,-118.166,0,NaN,NaN,NaN


In [9]:
# STEP 5: SEARCH ZILLOW AND GET ZESTIMATE, BEDROOMS, & BATHROOMS
# fill the dataframe with the data

search_zillow(la_df)
la_df

row 0: 3801 Sutro Ave Los Angeles CA 90008
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=3801%20Sutro%20Ave&citystatezip=%20Los%20Angeles%20CA%2090008
zestimate (value): 764746
bedrooms: 2
bathrooms: 2.0



row 1: 739 Fremont Villas Los Angeles CA 90042
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=739%20Fremont%20Villas&citystatezip=%20Los%20Angeles%20CA%2090042
zestimate (value): 534876
bedrooms: 2
bathrooms: 2.0



row 2: 1907 Redcliff St Los Angeles CA 90039
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=1907%20Redcliff%20St&citystatezip=%20Los%20Angeles%20CA%2090039
not for sale


row 3: 1206 W 36th St Los Angeles CA 90007
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=1206%20W%2036th%20St&citystatezip=%20Los%20Angeles%20CA%2090007
zestimate (value): 809670
bedrooms: 3
bathrooms

row 76: 2458 Flower St Huntington Park CA 90255
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=2458%20Flower%20St&citystatezip=%20Huntington%20Park%20CA%2090255
zestimate (value): 581355
bedrooms: 4
bathrooms: 4.0



row 77: 2406 Tesla Terrace Los Angeles CA 90039
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=2406%20Tesla%20Terrace&citystatezip=%20Los%20Angeles%20CA%2090039
zestimate (value): 1882837
bedrooms: 4
bathrooms: 2.0



row 79: 425S S Windsor Blvd Los Angeles CA 90020
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=425S%20S%20Windsor%20Blvd&citystatezip=%20Los%20Angeles%20CA%2090020
zestimate (value): 4454606
bedrooms: 5
bathrooms: 3.0



row 80: 874 W 40th Pl Los Angeles CA 90037
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=874%20W%2040th%20Pl&citystatezip=%20Los%20Angeles

row 139: 2809 Randolph St Huntington Park CA 90255
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=2809%20Randolph%20St&citystatezip=%20Huntington%20Park%20CA%2090255
zestimate (value): 411617
bedrooms: 2
bathrooms: 1.0



row 144: 3584 Tacoma Ave Los Angeles CA 90065
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=3584%20Tacoma%20Ave&citystatezip=%20Los%20Angeles%20CA%2090065
not for sale


row 146: 2911 S Hobart Blvd Los Angeles CA 90018
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=2911%20S%20Hobart%20Blvd&citystatezip=%20Los%20Angeles%20CA%2090018
zestimate (value): 606691
bedrooms: 3
bathrooms: 1.5



row 147: 5921 S San Pedro St Los Angeles CA 90003
https://www.zillow.com/webservice/GetDeepSearchResults.htm?zws-id=X1-ZWz18op3r67qq3_ahuyo&address=5921%20S%20San%20Pedro%20St&citystatezip=%20Los%20Angeles%20CA%2090003
zestimate (valu

,address,city_state_zip,latitude,longitude,message_code,zestimate,zillow_bathrooms,zillow_bedrooms
0,3801 Sutro Ave,Los Angeles CA 90008,34.0177,-118.324,0,764746,2.0,2
1,739 Fremont Villas,Los Angeles CA 90042,34.0965,-118.183,0,534876,2.0,2
2,1907 Redcliff St,Los Angeles CA 90039,34.0943,-118.27,0,NaN,NaN,NaN
3,1206 W 36th St,Los Angeles CA 90007,34.0227,-118.295,0,809670,2.0,3
7,443 46th St,Los Angeles CA 90011,34.0018,-118.268,0,422864,1.0,4
9,715 E 78th St,Los Angeles CA 90001,33.9686,-118.262,0,331920,1.0,2
15,3027 W 12th Pl,Los Angeles CA 90006,34.0486,-118.307,0,924100,3.0,5
16,3525 E 53rd St,Maywood CA 90270,33.9948,-118.202,0,432302,2.0,2
17,2312 Norwalk Ave,Los Angeles CA 90041,34.1328,-118.219,0,960272,3.0,3
18,4620 Eugene St,East Los Angeles CA 90022,34.0375,-118.166,0,482748,2.0,3


In [15]:
# do any further data cleaning you need to yourself
# for example, dropping any rows with NaN values
la_df = la_df.dropna(axis=0, how='any')
la_df

# maybe write to CSV to store the data for usage later/before doing plots? so you don't have to rerun everything

,address,city_state_zip,latitude,longitude,message_code,zestimate,zillow_bathrooms,zillow_bedrooms
0,3801 Sutro Ave,Los Angeles CA 90008,34.0177,-118.324,0,764746,2.0,2
1,739 Fremont Villas,Los Angeles CA 90042,34.0965,-118.183,0,534876,2.0,2
3,1206 W 36th St,Los Angeles CA 90007,34.0227,-118.295,0,809670,2.0,3
7,443 46th St,Los Angeles CA 90011,34.0018,-118.268,0,422864,1.0,4
9,715 E 78th St,Los Angeles CA 90001,33.9686,-118.262,0,331920,1.0,2
15,3027 W 12th Pl,Los Angeles CA 90006,34.0486,-118.307,0,924100,3.0,5
16,3525 E 53rd St,Maywood CA 90270,33.9948,-118.202,0,432302,2.0,2
17,2312 Norwalk Ave,Los Angeles CA 90041,34.1328,-118.219,0,960272,3.0,3
18,4620 Eugene St,East Los Angeles CA 90022,34.0375,-118.166,0,482748,2.0,3
29,4661 Palmero Dr,Los Angeles CA 90065,34.1152,-118.215,0,1267659,4.0,4
